# Core10_06 — Allocation Policy DSL (Deterministic)

## 목적
Core 10의 목적은 "설계 종료 이후에도 운영이 지속된다"를 운영 정책(배치 규칙)으로 증명하는 것이다.

본 노트북은 성능/최적화를 금지하고, 상태·궤적 기반의 **결정론적 배치 정책(DSL)** 을 정의한다.

## 금지
- 성능 기반 선택 금지
- 최적화(Optimization) 금지
- 학습 모델이 배치를 결정하는 것 금지
- LLM 판단으로 배치 결정 금지

## 허용
- 운영 생존성(operational survivability) 기반 정렬
- SoMS trajectory 안정성 기반 정렬
- 토글/진동 기반 정렬
- 다양성(미래 설계 옵션 보존) 기반 tie-break
- 모든 결정을 로그/규칙으로 설명 가능하게 유지

## 산출물
- core10_06_allocation_policy.json
  - Eligibility gate (운영 자격)
  - Ranking (정렬 규칙)
  - Tie-break (다양성/미래 설계 옵션 보존)
  - Cooldown / TTL / policy version

(참고) 본 노트북은 정책 계약을 만드는 것이며, 실제 배치 실행/시뮬레이션은 core10_07에서 수행한다.

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

ART_CORE10 = Path("../artifact/core10")
ART_CORE8  = Path("../artifact/core8")
ART_CORE9  = Path("../artifact/core9")

SCORES_PATH   = ART_CORE10 / "core10_04_survivability_scores.csv"   # fallback_pool + baselines
TRACE_PATH    = ART_CORE8  / "core8_03_refusal_state_trace_counterfactual.csv"  # SoMS/toggle/block/veto time series
RESLOG_PATH   = ART_CORE9  / "core9_04_reservation_log.csv"          # reservation (optional)
FALLBACK_PATH = ART_CORE8  / "core8_06_fallback_decisions.csv"       # fallback stage (optional)

OUT_DIR  = ART_CORE10
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_JSON = OUT_DIR / "core10_06_allocation_policy.json"

print("SCORES_PATH :", SCORES_PATH.resolve())
print("TRACE_PATH  :", TRACE_PATH.resolve())
print("RESLOG_PATH :", RESLOG_PATH.resolve())
print("FALLBACK    :", FALLBACK_PATH.resolve())
print("OUT_JSON    :", OUT_JSON.resolve())

assert SCORES_PATH.exists(), f"Missing: {SCORES_PATH.resolve()}"
assert TRACE_PATH.exists(),  f"Missing: {TRACE_PATH.resolve()}"

scores = pd.read_csv(SCORES_PATH)
trace  = pd.read_csv(TRACE_PATH)

reslog   = pd.read_csv(RESLOG_PATH) if RESLOG_PATH.exists() else None
fallback = pd.read_csv(FALLBACK_PATH) if FALLBACK_PATH.exists() else None

print("loaded:", {
    "scores": len(scores),
    "trace": len(trace),
    "reservation": None if reslog is None else len(reslog),
    "fallback": None if fallback is None else len(fallback),
})

scores.head(3)

SCORES_PATH : /Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_04_survivability_scores.csv
TRACE_PATH  : /Users/mac/Desktop/De/Developability_Data/core/artifact/core8/core8_03_refusal_state_trace_counterfactual.csv
RESLOG_PATH : /Users/mac/Desktop/De/Developability_Data/core/artifact/core9/core9_04_reservation_log.csv
FALLBACK    : /Users/mac/Desktop/De/Developability_Data/core/artifact/core8/core8_06_fallback_decisions.csv
OUT_JSON    : /Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_06_allocation_policy.json
loaded: {'scores': 19, 'trace': 180, 'reservation': 180, 'fallback': 180}


,antibody_id,signature,core10_operational_risk,proxy_survivability_score,tie_break_risk,logit_safe_prob,hazard_logit,tree_safe_prob,hazard_tree,hc_subtype,lc_subtype,cluster_size
0,GDPa1-060,IgG1|Kappa|0,0.156673,0.858995,1.308744,0.873860,0.126140,1.0,LOW,IgG1,Kappa,54
1,GDPa1-085,IgG1|Kappa|4,0.172660,0.844606,1.234429,0.919651,0.080349,1.0,LOW,IgG1,Kappa,49
2,GDPa1-021,IgG1|Kappa|1,0.172660,0.844606,1.244671,0.842986,0.157014,1.0,LOW,IgG1,Kappa,49


In [2]:
REQ_SCORES = [
    "antibody_id",
    "signature",
    "proxy_survivability_score",   # rule gate score (0~1)
    "tie_break_risk",              # deterministic tie-break risk (lower better)
    "core10_operational_risk",      # composite operational risk (lower better)
]
missing = [c for c in REQ_SCORES if c not in scores.columns]
assert not missing, f"Missing in scores: {missing}"

# numeric normalize
for c in ["proxy_survivability_score","tie_break_risk","core10_operational_risk"]:
    scores[c] = pd.to_numeric(scores[c], errors="coerce")

scores["proxy_survivability_score"] = scores["proxy_survivability_score"].fillna(0).clip(0, 1)
scores["tie_break_risk"] = scores["tie_break_risk"].fillna(scores["tie_break_risk"].median())
scores["core10_operational_risk"] = scores["core10_operational_risk"].fillna(scores["core10_operational_risk"].median())

# trace numeric normalize
trace["step"] = pd.to_numeric(trace.get("step"), errors="coerce").astype("Int64")
trace["SoMS_cumsum_window"] = pd.to_numeric(trace.get("SoMS_cumsum_window"), errors="coerce")
trace["action_toggle_rate"] = pd.to_numeric(trace.get("action_toggle_rate"), errors="coerce")

# (선택) blocked_rate_window / veto_streak 있으면 정리
if "blocked_rate_window" in trace.columns:
    trace["blocked_rate_window"] = pd.to_numeric(trace["blocked_rate_window"], errors="coerce")
if "veto_streak" in trace.columns:
    trace["veto_streak"] = pd.to_numeric(trace["veto_streak"], errors="coerce")

print("scores cols:", scores.columns.tolist())
print("trace  cols:", trace.columns.tolist())

scores cols: ['antibody_id', 'signature', 'core10_operational_risk', 'proxy_survivability_score', 'tie_break_risk', 'logit_safe_prob', 'hazard_logit', 'tree_safe_prob', 'hazard_tree', 'hc_subtype', 'lc_subtype', 'cluster_size']
trace  cols: ['run_id', 'case_id', 'antibody_id', 'step', 'refusal_stage', 'refusal_mode', 'blocked_rate_window', 'veto_streak', 'action_toggle_rate', 'SoMS_cumsum_window', 'refusal_triggered', 'refusal_reason_code']


## Policy DSL 구조 (Core10_06_v1)

정책은 3단으로 구성한다.

### 1) Eligibility Gate (운영 자격)
- (필수) Refusal/Fallback Enter 상태면 운영 배치 제외
- (선택) Reservation CONFIRMED/ CANDIDATE 상태면 운영 “주의” 또는 제외
- (필수) Cooldown: 직전 운영 교체 직후 N step 동안은 강제 고정(토글 방지)

### 2) Ranking (성능 금지)
- survivability 최대 (proxy_survivability_score 높은 순)
- SoMS slope 최소 (최근 k step 추세)
- toggle streak 최소 (최근 k에서 토글 임계 초과 연속 길이)
- tie_break_risk 최소 (운영 타이브레이커 위험)

### 3) Tie-break (미래 설계 옵션 보존)
- signature(= hc|lc|cluster) 기반으로 다양성 유지
- 동일 signature에서 대표 1개만 우선 선택(운영 대기 풀 방식)
- 마지막 tie-break: antibody_id 사전순 (완전 결정론)

In [3]:
WINDOW_K = 5  # 최근 k-step 윈도우 (운영 친화적, 너무 작게 X)

TOGGLE_TH = 0.45  # Core8에서 쓰던 toggle 임계와 일관 유지 (필요시 조정)
TOGGLE_STREAK_K = WINDOW_K

SOMS_SLOPE_K = WINDOW_K

METRICS_SPEC = {
    "window_k": WINDOW_K,
    "toggle_threshold": TOGGLE_TH,
    "metrics": {
        "soms_slope_k": {
            "input": "SoMS_cumsum_window",
            "definition": "linear_slope over last k steps (per antibody trajectory)",
            "k": SOMS_SLOPE_K,
            "direction": "lower_better",
        },
        "toggle_over_th_streak_k": {
            "input": "action_toggle_rate",
            "definition": "consecutive streak length over last k steps where toggle_rate >= threshold",
            "k": TOGGLE_STREAK_K,
            "threshold": TOGGLE_TH,
            "direction": "lower_better",
        },
    }
}

METRICS_SPEC

{'window_k': 5,
 'toggle_threshold': 0.45,
 'metrics': {'soms_slope_k': {'input': 'SoMS_cumsum_window',
   'definition': 'linear_slope over last k steps (per antibody trajectory)',
   'k': 5,
   'direction': 'lower_better'},
  'toggle_over_th_streak_k': {'input': 'action_toggle_rate',
   'definition': 'consecutive streak length over last k steps where toggle_rate >= threshold',
   'k': 5,
   'threshold': 0.45,
   'direction': 'lower_better'}}}

In [4]:
POLICY_ID = "core10_06_allocation_policy_v1"

# 운영 토글 방지 cooldown
COOLDOWN_STEPS = 3  # "운영 중 교체 직후 N step 동안 강제 유지"

ELIGIBILITY = {
    "cooldown": {
        "enabled": True,
        "cooldown_steps": COOLDOWN_STEPS,
        "reason_code": "REASON_COOLDOWN_ACTIVE",
    },
    "refusal_exclusion": {
        "enabled": True,
        "exclude_if_fallback_stage_in": ["REFUSAL", "FALLBACK_ENTER"],  # Core8 의미 그대로
        "reason_code": "REASON_DESIGN_SHUTDOWN_EXCLUSION",
    },
    "reservation_handling": {
        "enabled": True if reslog is not None else False,
        "exclude_if_reservation_status_in": ["CONFIRMED"],   # confirmed는 “닫아야 한다” 신호 → 운영 후보에서 제외 가능
        "degrade_if_reservation_status_in": ["CANDIDATE"],   # 후보는 운영 가능하되 감점/주의
        "reason_code_exclude": "REASON_RESERVATION_CONFIRMED_EXCLUSION",
        "reason_code_degrade": "REASON_RESERVATION_CANDIDATE_DEGRADE",
    }
}

ELIGIBILITY

{'cooldown': {'enabled': True,
  'cooldown_steps': 3,
  'reason_code': 'REASON_COOLDOWN_ACTIVE'},
 'refusal_exclusion': {'enabled': True,
  'exclude_if_fallback_stage_in': ['REFUSAL', 'FALLBACK_ENTER'],
  'reason_code': 'REASON_DESIGN_SHUTDOWN_EXCLUSION'},
 'reservation_handling': {'enabled': True,
  'exclude_if_reservation_status_in': ['CONFIRMED'],
  'degrade_if_reservation_status_in': ['CANDIDATE'],
  'reason_code_exclude': 'REASON_RESERVATION_CONFIRMED_EXCLUSION',
  'reason_code_degrade': 'REASON_RESERVATION_CANDIDATE_DEGRADE'}}

In [5]:
RANKING = [
    {
        "name": "survivability_primary",
        "field": "proxy_survivability_score",
        "order": "desc",
        "reason_code": "RANK_SURVIVABILITY_MAX",
    },
    {
        "name": "soms_slope_secondary",
        "field": "soms_slope_k",
        "order": "asc",
        "reason_code": "RANK_SOMS_SLOPE_MIN",
    },
    {
        "name": "toggle_streak_tertiary",
        "field": "toggle_over_th_streak_k",
        "order": "asc",
        "reason_code": "RANK_TOGGLE_STREAK_MIN",
    },
    {
        "name": "tie_break_risk_quaternary",
        "field": "tie_break_risk",
        "order": "asc",
        "reason_code": "RANK_TIE_BREAK_RISK_MIN",
    },
    {
        "name": "deterministic_last",
        "field": "antibody_id",
        "order": "asc",
        "reason_code": "RANK_ID_DETERMINISTIC",
    }
]

# -------------------------
# Diversity / Future Design Option Preservation
# -------------------------
DIVERSITY = {
    "enabled": True,
    "signature_field": "signature",
    "rule": "prefer distinct signature reps before taking multiple from same signature",
    "max_per_signature": 1,
    "reason_code": "RULE_FUTURE_DESIGN_OPTION_PRESERVATION",
}

RANKING, DIVERSITY

([{'name': 'survivability_primary',
   'field': 'proxy_survivability_score',
   'order': 'desc',
   'reason_code': 'RANK_SURVIVABILITY_MAX'},
  {'name': 'soms_slope_secondary',
   'field': 'soms_slope_k',
   'order': 'asc',
   'reason_code': 'RANK_SOMS_SLOPE_MIN'},
  {'name': 'toggle_streak_tertiary',
   'field': 'toggle_over_th_streak_k',
   'order': 'asc',
   'reason_code': 'RANK_TOGGLE_STREAK_MIN'},
  {'name': 'tie_break_risk_quaternary',
   'field': 'tie_break_risk',
   'order': 'asc',
   'reason_code': 'RANK_TIE_BREAK_RISK_MIN'},
  {'name': 'deterministic_last',
   'field': 'antibody_id',
   'order': 'asc',
   'reason_code': 'RANK_ID_DETERMINISTIC'}],
 {'enabled': True,
  'signature_field': 'signature',
  'rule': 'prefer distinct signature reps before taking multiple from same signature',
  'max_per_signature': 1,
  'reason_code': 'RULE_FUTURE_DESIGN_OPTION_PRESERVATION'})

In [6]:
policy = {
    "policy_id": POLICY_ID,
    "core": "core10",
    "type": "allocation_policy_dsl",
    "version": "v1",
    "principles": {
        "no_performance_optimization": True,
        "deterministic": True,
        "no_llm_decision": True,
        "operation_continues_after_shutdown": True,
        "future_design_option_preservation": True,
    },
    "inputs": {
        "scores_csv": str(SCORES_PATH),
        "trace_csv": str(TRACE_PATH),
        "reservation_log_csv": str(RESLOG_PATH) if (reslog is not None) else None,
        "fallback_decisions_csv": str(FALLBACK_PATH) if (fallback is not None) else None,
    },
    "eligibility": ELIGIBILITY,
    "metrics_spec": METRICS_SPEC,
    "ranking": RANKING,
    "diversity": DIVERSITY,
    "outputs": {
        "artifact": "core10_06_allocation_policy.json",
        "note": "This artifact is imported by core10_07 allocation executor.",
    },
    "logging": {
        "required_fields": [
            "run_id","case_id","step",
            "selected_antibody_id",
            "eligibility_rejections",
            "ranking_fields_snapshot",
            "policy_id",
            "decision_reason_codes"
        ],
        "note": "Allocation executor must log deterministic reasons at each step."
    }
}

OUT_JSON.write_text(json.dumps(policy, indent=2), encoding="utf-8")
print("✅ Exported policy:", OUT_JSON.resolve())

✅ Exported policy: /Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_06_allocation_policy.json


In [7]:
preview_cols = ["antibody_id","signature","proxy_survivability_score","tie_break_risk","core10_operational_risk"]
tmp = scores[preview_cols].copy()

tmp = tmp.sort_values(
    ["proxy_survivability_score","tie_break_risk","antibody_id"],
    ascending=[False, True, True]
)

print("Top 10 (static policy fields only):")
tmp.head(10)

Top 10 (static policy fields only):


,antibody_id,signature,proxy_survivability_score,tie_break_risk,core10_operational_risk
0,GDPa1-060,IgG1|Kappa|0,0.858995,1.308744,0.156673
1,GDPa1-085,IgG1|Kappa|4,0.844606,1.234429,0.172660
2,GDPa1-021,IgG1|Kappa|1,0.844606,1.244671,0.172660
3,GDPa1-025,IgG1|Lambda|4,0.844606,1.335000,0.172660
4,GDPa1-165,IgG1|Lambda|1,0.844606,1.350459,0.172660
5,GDPa1-017,IgG1|Kappa|2,0.841369,1.154242,0.176257
6,GDPa1-010,IgG1|Kappa|3,0.834472,1.336067,0.183920
7,GDPa1-138,IgG4|Kappa|0,0.505395,1.335473,0.549561
8,GDPa1-050,IgG4|Kappa|4,0.491007,1.426338,0.565548
9,GDPa1-039,IgG4|Kappa|1,0.491007,1.510580,0.565548


## 심사용 고정 문구 (Core10_06)

- “Core10_06은 성능을 최적화하지 않는다. 운영 지속성을 위해 결정론적 배치 정책(DSL)을 정의한다.”
- “배치 정책은 Eligibility Gate → Ranking → Diversity-preserving Tie-break의 3단 구조로 고정된다.”
- “모든 선택은 policy_id와 reason_code로 설명 가능하며, 운영 로그로 영속화 가능하다.”